In [1]:
from pandas import read_csv, Series
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
from pickle import load
from glove import Glove
import adagram
from gensim.models.wrappers import FastText, Wordrank
from embed_utils import Word2VecF, Swivel, cosine_sim, get_adagram_sense_prob, wv
from utils.string_utils import morph_parse, make_tokens

In [2]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.dummy import DummyClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import (accuracy_score, brier_score_loss, precision_score, recall_score, f1_score)
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [88]:
import matplotlib.pyplot as plt
import seaborn

In [4]:
old_err_state = np.seterr(all='raise')

def vectorize_message(sentence, model, num_features, vocab):
    featureVec = np.zeros((num_features), dtype='float32')
    nwords = 0
    
    tokens = make_tokens(sentence.lower(), vocab)

    for word in tokens:
        if word in vocab: 
            if model == 'word2vec':
                featureVec = np.add(featureVec, word2vec[word])
            elif model == 'wang2vec':
                featureVec = np.add(featureVec, wang2vec[word])
            elif model == 'glove':
                featureVec = np.add(featureVec, wv(glove, word))
            elif model == 'word2vecf':
                featureVec = np.add(featureVec, w2vf.word2vec(word))
            elif model == 'adagram':
                featureVec = np.add(featureVec, ada_model.sense_vector(word, get_adagram_sense_prob(ada_model, word)))
            elif model == 'fasttext':
                featureVec = np.add(featureVec, ft[word])
            elif model == 'bow':
                featureVec = np.add(featureVec, bow[word])
            elif model == 'swivel':
                featureVec = np.add(featureVec, swivel.lookup(word))
            nwords = nwords + 1
    try:
        featureVec = np.divide(featureVec, nwords)
    except FloatingPointError:
         featureVec = np.zeros((num_features), dtype='float32')
    return featureVec

In [5]:
df = read_csv('anno2ch/annotated.csv', encoding='cp1251')
df.comment = df.comment.apply(morph_parse)
df.reference = df.reference.apply(morph_parse)

In [6]:
df_unl = read_csv('anno2ch/annotated.csv', encoding='cp1251')

In [7]:
Y = df['labels'].values

Загрузка Word2Vec-модели

In [8]:
word2vec = Word2Vec.load('models/word2vec/all_lem_100')
word2vec_vocab = word2vec.wv.vocab

Загрузка Glove-модели

In [9]:
with open('models/glove/all_lem_100', 'rb') as fp:
    glove = load(fp)
glove_vocab = glove.dictionary

Загрузка Wang2Vec-модели

In [10]:
wang2vec = KeyedVectors.load_word2vec_format('models/wang2vec/all_lem_100_cwindow', binary=True)
wang2vec_vocab = wang2vec.vocab

Загрузка Word2Vec-f-модели

In [11]:
from os import path

w2vf = Word2VecF.load(path.join('models/word2vecf', 'vecs.npy'), path.join('models/word2vecf', 'vecs.vocab'))
w2vf_vocab = w2vf._vocab

Загрузка Adagram-модели

In [12]:
ada_model = adagram.VectorModel.load('models/adagram/all_lem_100.pkl')
adagram_vocab = ada_model.dictionary.word2id.keys()

Загрузка BOW-модели

In [13]:
from pickle import load

with open('models/tf_idf/all_lem', 'rb') as fp:
    bow = load(fp)

Загрузка Swivel-модели

In [14]:
swivel = Swivel('models/swivel/vocab_100.txt', 'models/swivel/vecs_100.bin')
swivel_vocab = swivel.vocab

Загрузка Fasttext-модели

In [15]:
ft = FastText.load_word2vec_format('models/fasttext/all_100_skipgram.vec')
ft_vocab = ft.vocab

Загрузка Wordrank-модели

In [16]:
#wr = Wordrank()
#mod = Wordrank.load_wordrank_model('/media/defeater/d5233b78-d9c7-40e7-a454-8b108bcc4b8a/defeater/NLP/wordrank/meta/cooccurrence')
#wr = wr.train('/media/defeater/d5233b78-d9c7-40e7-a454-8b108bcc4b8a/defeater/NLP/wordrank', corpus_file='corpus.txt', out_path='/media/defeater/d5233b78-d9c7-40e7-a454-8b108bcc4b8a/defeater/NLP/wordrank')

Получение датасетов

In [17]:
word2vec_vectors = np.zeros(shape=(len(df),100), dtype='float32')

for i, m in df.iterrows():
    word2vec_vectors[i] = vectorize_message(m['comment'] + m['reference'], 'word2vec', 100, word2vec_vocab)

In [18]:
glove_vectors = np.zeros(shape=(len(df),100), dtype='float32')

for i, m in df.iterrows():
    glove_vectors[i] = vectorize_message(m['comment'] + m['reference'], 'glove', 100, glove_vocab)

In [19]:
wang2vec_vectors = np.zeros(shape=(len(df),100), dtype='float32')

for i, m in df.iterrows():
    wang2vec_vectors[i] = vectorize_message(m['comment'] + m['reference'], 'wang2vec', 100, wang2vec_vocab)

In [20]:
adagram_vectors = np.zeros(shape=(len(df),100), dtype='float32')

for i, m in df.iterrows():
    adagram_vectors[i] = vectorize_message(m['comment'] + m['reference'], 'adagram', 100, adagram_vocab)

In [21]:
w2vf_vectors = np.zeros(shape=(len(df),100), dtype='float32')

for i, m in df_unl.iterrows():
    w2vf_vectors[i] = vectorize_message(m['comment'] + m['reference'], 'word2vecf', 100, w2vf_vocab)

In [22]:
ft_vectors = np.zeros(shape=(len(df),100), dtype='float32')

for i, m in df.iterrows():
    ft_vectors[i] = vectorize_message(m['comment'] + m['reference'], 'fasttext', 100, ft_vocab)

In [23]:
bow_vectors = np.zeros(shape=(len(df),1), dtype='float32')

for i, m in df.iterrows():
    bow_vectors[i] = vectorize_message(m['comment'] + m['reference'], 'bow', 1, bow)

In [24]:
swivel_vectors = np.zeros(shape=(len(df), 100), dtype='float32')

for i, m in df.iterrows():
    swivel_vectors[i] = vectorize_message(m['comment'] + m['reference'], 'swivel', 100, swivel_vocab)

Сравнение

In [90]:
def set_plt_params():
    title_font = {'size':'80', 'color':'black', 'weight':'bold',
                  'verticalalignment':'bottom'} 
    axis_font = {'size':'80'}
    plt.figure(figsize=(10, 10))
    #plt.xlabel('Training examples in cross-validation', **axis_font)
    #plt.ylabel('F1-Score',  **axis_font)
    plt.suptitle('(a) Summation', fontsize=90)
    plt.grid(False)
    plt.axes(frameon = 0)
    plt.tick_params(labelsize=60)
    plt.ylim([0.76, 0.86])
    #plt.setp(fontsize='large')
    #seaborn.despine()

In [91]:
set_plt_params()
results = []

import pylab

figlegend = pylab.figure(figsize=(3,2))

seaborn.set_style('white', {'grid' : False})

lss = [
]
names = []
for X, name, lstyle in [(glove_vectors, 'Glove', 'dotted'),
                (word2vec_vectors, 'Word2Vec', 'dashed'),
                (wang2vec_vectors, 'Wang2Vec', 'dashdot'),
                (w2vf_vectors, 'Word2Vec-f',  (0, (20,20))),
                (adagram_vectors, 'Adagram', (0,(10,10))),
                (ft_vectors, 'FastText','solid'),
                (bow_vectors, 'TF-IDF',(0,(30,30))),
                (swivel_vectors, 'Swivel',(0,(40,40))),
                ]:
    estimator = KNeighborsClassifier(n_neighbors = 3)
    cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
    train_sizes=np.linspace(0.1, 0.9, 5)
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, Y, cv=cv, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    results.append({'model' : name, 'score' : train_scores_mean})
    plt.plot(train_sizes, train_scores_mean, label=name, linewidth=10, linestyle=lstyle)

    
plt.legend(prop={'size':60}, ncol=8)
plt.savefig('classifiers_legend.png', bbox_inches='tight')
#plt.show()

In [37]:
results

[{'model': 'Glove',
  'score': array([ 0.80294118,  0.82658537,  0.82865497,  0.83458725,  0.84443542])},
 {'model': 'Word2Vec',
  'score': array([ 0.79117647,  0.82292683,  0.83055556,  0.83563218,  0.83744923])},
 {'model': 'Wang2Vec',
  'score': array([ 0.80808824,  0.81878049,  0.8371345 ,  0.84242424,  0.84630382])},
 {'model': 'Word2Vec-f',
  'score': array([ 0.75588235,  0.77926829,  0.77646199,  0.77826541,  0.77912266])},
 {'model': 'Adagram',
  'score': array([ 0.78014706,  0.81      ,  0.82017544,  0.82361546,  0.82493907])},
 {'model': 'FastText',
  'score': array([ 0.8125    ,  0.83780488,  0.84415205,  0.84963427,  0.85442729])},
 {'model': 'TF-IDF',
  'score': array([ 0.76470588,  0.77536585,  0.77017544,  0.76781609,  0.76856214])},
 {'model': 'Swivel',
  'score': array([ 0.80735294,  0.83487805,  0.84517544,  0.8539185 ,  0.85686434])}]